In [89]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8 # bigram model parameter
batch_size = 4 # how we will stack the blocks
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [9]:
data_location = "/mnt/a/LLM_exploration_DB/wizard_of_oz.txt"

In [78]:
with open(data_location, 'r', encoding='utf8') as f:
    text = f.read()
# create a set of all characters
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [11]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [12]:
#encoded_hello = encode('hello')
#decoded_hello = decode(encoded_hello)
#print(decoded_hello)

hello


In [13]:
data = torch.tensor(encode(text), dtype=torch.long) # long sequence of integers

In [15]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [82]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device) # push to GPU
    return x,y


In [68]:
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([ 52693, 175643, 112244,  14355])
inputs:
tensor([[ 0, 48, 65, 62,  1, 70, 58, 66],
        [ 1, 63, 62, 58, 77, 78, 75, 62],
        [77, 65, 62,  1, 60, 65, 66, 69],
        [75, 75, 66, 59, 69, 62,  1, 76]], device='cuda:0')
targets:
tensor([[48, 65, 62,  1, 70, 58, 66, 61],
        [63, 62, 58, 77, 78, 75, 62, 76],
        [65, 62,  1, 60, 65, 66, 69, 61],
        [75, 66, 59, 69, 62,  1, 76, 62]], device='cuda:0')


In [18]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is ", context, 'target is ', target)

when input is  tensor([91]) target is  tensor(48)
when input is  tensor([91, 48]) target is  tensor(65)
when input is  tensor([91, 48, 65]) target is  tensor(62)
when input is  tensor([91, 48, 65, 62]) target is  tensor(1)
when input is  tensor([91, 48, 65, 62,  1]) target is  tensor(44)
when input is  tensor([91, 48, 65, 62,  1, 44]) target is  tensor(75)
when input is  tensor([91, 48, 65, 62,  1, 44, 75]) target is  tensor(72)
when input is  tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is  tensor(67)


In [79]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__() # initialize things from nn.Module
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # this is basically a lookup table

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) # 3 dim logits when target is none

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # batch, time, channels
            logits = logits.view(B*T, C) # works with logits have batch and time. B and T are combined to N so that the output is N,C
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indicies in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)# (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device) # torch.long is int64
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Tw$zN? c:‘-76"Jt”BM&TAp4vST—S?[s'xq$&H,7BTw&p%dh_0;:3C8i.,5Udh“6$TiY”mmu
oX&eeKv4]MpEaO]y&—0;-l*qlI;9kLah-A.‘‘*_E—5OlBr7A5a5JdNV1pg﻿KrNBotN?3?‘-qb’GxaM.y
(/(F™•“ F $U—uK)]1﻿.Jt“hU﻿-[M’oPTA"YU3i$?7s_![M Kh'e15dOZ0•9q—,KpH“”2uD['9g9’S
1A•uP——‘9#Csd1A9'D0%1;’﻿*ku?XGRYab[VEBv‘]4nAt;z:-yeGx‘ocFymE_q8(_—D﻿2
’NTg?kbnyJt2N[U)g—r﻿—ub’/np5U;Jg!“iU[f”PiB';qLI‘4t7f'8P.,KMSjNbJ[6*—qG._"6R™K1[u?pIQ,wP”:Qgt#,5F1Z;nm;6•HF KwR7&.yM$?AQ﻿F2K
JMst'lW],s9'YgC0SlwR7f9oOMPZc”Pm;NxbY2r_pk﻿*J[Z$-i'7MUnTYi7cF999a dK7yf2u


In [92]:
@torch.no_grad() # make sure no gradient is used here. model is passed into it so could be an issue
def estimate_loss():
    out = {}
    model.eval() # switch to eval
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # switch back to train
    return out

In [98]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters ==0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # dont add gradients over time. keep on for RNNs.
    loss.backward()
    optimizer.step()
print (loss.item())

step: 0, train loss: 2.426, val loss: 2.648
step: 250, train loss: 2.414, val loss: 2.657
step: 500, train loss: 2.432, val loss: 2.696
step: 750, train loss: 2.439, val loss: 2.680
step: 1000, train loss: 2.423, val loss: 2.667
step: 1250, train loss: 2.435, val loss: 2.676
step: 1500, train loss: 2.450, val loss: 2.705
step: 1750, train loss: 2.433, val loss: 2.628
step: 2000, train loss: 2.428, val loss: 2.665
step: 2250, train loss: 2.425, val loss: 2.621
step: 2500, train loss: 2.430, val loss: 2.647
step: 2750, train loss: 2.429, val loss: 2.653
step: 3000, train loss: 2.443, val loss: 2.676
step: 3250, train loss: 2.420, val loss: 2.645
step: 3500, train loss: 2.411, val loss: 2.661
step: 3750, train loss: 2.443, val loss: 2.664
step: 4000, train loss: 2.429, val loss: 2.666
step: 4250, train loss: 2.446, val loss: 2.675
step: 4500, train loss: 2.410, val loss: 2.689
step: 4750, train loss: 2.433, val loss: 2.646
step: 5000, train loss: 2.425, val loss: 2.638
step: 5250, train l

In [96]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


s. lig a ate ald ied s theatrs vintacarind we oute arerronimprd t by aro
"Pys
russt flk minomameve ndd areacotis d.   leain I'lso  th,"ARF“_ ithe buithathais ftan eed h a meshin hed nss sed Ho t qure thrasm ar wht l, t brs hewnime t asagga stle."
tlleanered tofithean, aifforeand y, ourt mpure bel, bu to fe gof  initoveve; e fot PR."

plise angancon. ce," qully, s wh ardoun thiny E bace te  tenyeslly th walifo a nord "I I greadasonglled y ason's, pin he,

o otuntldvenved loond kn Pre aid the Jer 
